# LCEL 파이프 연산자(|)로 선언적 체인 구성하기

이 노트북에서는 **LCEL(LangChain Expression Language)**의 **파이프 연산자(|)**를 사용하여 선언적(Declarative) 방식으로 체인을 구성하는 방법을 알아봅니다.

## LCEL이란?

LangChain Expression Language는 **파이프 연산자(|)**를 사용하여 컴포넌트를 연결하는 선언적 문법입니다.

```python
chain = component_a | component_b | component_c
```

Unix의 파이프(`|`)처럼, 앞 컴포넌트의 **출력**이 다음 컴포넌트의 **입력**이 됩니다.

## 선언적 vs 명령적

| 방식 | 코드 스타일 | 특징 |
|------|------------|------|
| **선언적 (LCEL)** | `chain = a \| b \| c` | "무엇을" 연결할지 선언 |
| **명령적 (@chain)** | 함수 내부에 로직 작성 | "어떻게" 실행할지 명시 |

## LCEL의 장점

1. **간결함**: 한 줄로 체인 구성
2. **가독성**: 데이터 흐름이 명확
3. **자동 최적화**: 스트리밍, 배치, 비동기 자동 지원
4. **조합 용이**: 체인을 더 큰 체인의 부품으로 사용

## 실행 흐름

```
입력값 {'question': '...'}
         │
         ▼

ChatPromptTemplate  → 메시지 리스트 생성

         │
         ▼

    ChatOllama     → LLM 호출

         │
         ▼
    AIMessage 응답
```

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (853 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current use

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. 구성 요소 준비

체인에서 사용할 **Prompt Template**과 **Model**을 정의합니다.

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# Prompt Template
template = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 어시스턴트입니다.'),
    ('human', '{question}'),
])

# Model
model = ChatOllama(model='llama3.2')

# 4. LCEL로 체인 구성

**핵심 코드:**

```python
chatbot = template | model
```

단 한 줄로 체인이 완성됩니다!

### 파이프 연산자(|) 동작 원리

1. `template`의 출력 (ChatPromptValue) →
2. `model`의 입력으로 전달 →
3. `model`의 출력 (AIMessage) 반환

### @chain 방식과 비교

```python
# LCEL (선언적) - 이 노트북
chatbot = template | model

# @chain (명령적) - 11번 노트북
@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)
```

결과는 동일하지만, LCEL이 훨씬 간결합니다.

In [4]:
# LCEL로 체인 구성 - 단 한 줄!
chatbot = template | model

print(f"체인 타입: {type(chatbot).__name__}")

체인 타입: RunnableSequence


# 5. 체인 실행

LCEL로 만든 체인도 **Runnable 인터페이스**를 갖습니다.

- `chatbot.invoke()` - 단일 실행
- `chatbot.batch()` - 배치 실행
- `chatbot.stream()` - 스트리밍

In [5]:
# invoke() 실행
response = chatbot.invoke({'question': '거대 언어 모델은 어디서 제공하나요?'})

print("=== invoke() 결과 ===")
print(response.content)

=== invoke() 결과 ===
지금까지 कई 곳에서 대규모 언어 모델을 개발하고 사용했습니다.

1. **Google**: Google의 Language Model을 통해 Large Language Model을 개발했습니다.
2. **Hugging Face**: Hugging Face는 Large Language Model을 개발하고 공개했습니다. 
3. **OpenAI**: OpenAI도 Large Language Model을 개발하고 공개했습니다.


In [6]:
# batch() 실행
responses = chatbot.batch([
    {'question': 'Python이 뭔가요?'},
    {'question': 'LangChain이 뭔가요?'},
])

print("=== batch() 결과 ===")
for i, resp in enumerate(responses):
    print(f"\n[{i+1}] {resp.content[:100]}...")

=== batch() 결과 ===

[1] Python은 일종의 프로그래밍 언어로, 자세한 것을 swift, java 등과는 다르며, Python을 사용할 때, 개발자들은 code를 작성하고, 컴퓨터에 lệnh을 줄 수 있...

[2] LangChain은 LangChain에 의해서 개발된 인공 지능 언어 모델입니다. 

langchain은 LangChain에 의해서 개발된 인공 지능 언어 모델입니다....


# 6. 더 긴 체인 구성

LCEL은 여러 컴포넌트를 자유롭게 연결할 수 있습니다.

In [7]:
from langchain_core.output_parsers import StrOutputParser

# 3단계 체인: Prompt → Model → OutputParser
chain_with_parser = template | model | StrOutputParser()

# 이제 결과가 AIMessage가 아닌 str
result = chain_with_parser.invoke({'question': '안녕하세요!'})

print("=== StrOutputParser 추가 ===")
print(f"타입: {type(result).__name__}")
print(f"결과: {result}")

=== StrOutputParser 추가 ===
타입: str
결과: 안녕하세요! 나에대해 좋지 않는อะไร 없으신가요?


# 7. 체인 시각화

LCEL 체인의 구조를 확인할 수 있습니다.

In [8]:
# 체인 구조 출력
print("=== 체인 구조 ===")
print(chain_with_parser)

=== 체인 구조 ===
first=ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 어시스턴트입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]) middle=[ChatOllama(model='llama3.2')] last=StrOutputParser()


---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI()

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## LCEL 체인 패턴 모음

### 기본 체인
```python
chain = prompt | model
```

### 출력 파서 추가
```python
chain = prompt | model | StrOutputParser()
```

### Structured Output
```python
chain = prompt | model.with_structured_output(MySchema)
```

### 여러 체인 조합
```python
chain1 = prompt1 | model
chain2 = prompt2 | model
combined = chain1 | chain2  # 체인을 체인에 연결
```

## 관련 노트북 정리

| 노트북 | 방식 | 키워드 |
|--------|------|--------|
| 11.imperative | 명령적 | `@chain`, `def` |
| 12.stream | 명령적 + 스트리밍 | `@chain`, `yield` |
| 13.async | 명령적 + 비동기 | `@chain`, `async def` |
| **14.declarative** | **선언적** | **`\|` 연산자** |
| 15.declarative-stream | 선언적 + 스트리밍 | `\|`, `stream()` |